# Задача №1

🤔 <font color='brown'> Провести частичный анализ OVAL файла от компании Red Hat для ОС
RHEL8 (https://www.redhat.com/security/data/oval/v2/RHEL8/rhel8.oval.xml.bz2) на первых 3 уязвимостях (патчах) и определить набор
объектов, из которых он состоит. Разобрать основную логику работы с
данным форматом. </font> 

### 🚀 Основная логика:

<font color='green'><b>&lt;definitions&gt;</b></font> содержит два основных тега: <font color='brown'><b>&lt;metadata&gt;</b></font> и <font color='brown'><b>&lt;criteria&gt;</b></font>, где первый - для информационных целей, второй - критерии и проверки для определения этой уязвимости, с ссылающимися на тесты внутри объекта <font color='purple'><b>&lt;tests&gt;</b></font>. Информация внутри объекта <font color='purple'><b>&lt;tests&gt;</b></font> служит для связи <font color='red'><b>&lt;objects&gt;</b></font> (объекты) и <font color='orange'><b>&lt;states&gt;</b></font> (состояния) вместе. Тест считается пройденным, когда ресурс оцениваемой системы, обозначаемый данным элементом объекта, соответствует данному элементу состояния.

🤔 <font color='brown'> Описать кратко текстом объекты, которые были найдены и для чего они
используются.</font> 

### 🚀 Кратко об объектах
- <font color='blue'><b>&lt;generator&gt;</b></font> – информация о происхождении документа и различная дополнительная информация, связанная с его содержимым.

- <font color='green'><b>&lt;definitions&gt;</b></font> – содержит общую информацию об уязвимостях и критериях их определения.

- <font color='purple'><b>&lt;tests&gt;</b></font> – служит для связи объектов и состояний вместе. Главная задача <font color='purple'><b>&lt;tests&gt;</b></font> - сопоставление текущего состояния объекта с требуемым.

- <font color='red'><b>&lt;objects&gt;</b></font> – предоставляет сведения, определяющие уникальный набор совпадающих элементов.

- <font color='orange'><b>&lt;states&gt;</b></font> – задает требуемое состояние объекта. Состояние может быть задано статически или определяться через переменную.

- <font color='brown'><b>&lt;variables&gt;</b></font> <i>(в данном наборе данных не используется)</i> – определяет внешние переменные, которые можно использовать для настройки элементов из раздела <font color='orange'><b>&lt;states&gt;</b></font>.

🤔 <font color='brown'> В рамках каждой «уязвимости», есть условия по её выявлению: какие
на ваш взгляд из критериев лишние, а какие обязательны? </font> 

### 🚀 Учёт критериев в анализе

#### На мой взгляд лишение три вида критериев (в скрипте я их учитывать не буду):


1) Критерии, говорящие о присутствии модуля в ОС
    - Данный критерий является избыточным, так как не несет конкретной информации, а носит больше информативный характер.  
    - Буду рассматривать точные критерии, которые указывают на объекты и состояния, а они, в свою очередь, имеют точное название пакета + версии (или порога версии).  
    - 📌 Пример:  
    ```xml
<criterion comment="Module ruby:2.5 is enabled" test_ref="oval:com.redhat.rhba:tst:20193384065"/>
```

2) Критерии, говорящие о присутствии ключа безопасности
    - Red Hat использует в своих продуктах ключ безопасности для подтверждения подлинности пакета - **redhatrelease2 key**. 
    - 📌 Пример:  
    ```xml
<criterion comment="ruby-devel is signed with Red Hat redhatrelease2 key" test_ref="oval:com.redhat.rhba:tst:20193384004"/>
```

3) Критерии, говорящие что RHEL должен быть установлен:
    - 📌 Пример: 
    ```xml
<criterion comment="Red Hat Enterprise Linux must be installed" test_ref="oval:com.redhat.rhba:tst:20191992005"/>
```


#### Обязательные критерии - все остальные:
1) Критерии, указывающие, что конкретный продукт установлен
    - Хоть документ xml изначально предназначался для RHEL 8, 20 арпеля 2021 года был добален второй продукт - Red Hat CoreOS 4.
    - Чтобы не пропустить добавления новых продуктов, проверка обязательна.
    - 📌 Пример:  
    ```xml
<criterion comment="Red Hat Enterprise Linux 8 is installed" test_ref="oval:com.redhat.rhba:tst:20191992003"/>
# OR
<criterion comment="Red Hat CoreOS 4 is installed" test_ref="oval:com.redhat.rhba:tst:20191992004"/>
```

2) Критерии с конкретным пакетом
    - Указывают на тесты с конкретным пакетом + версией (или порогом версии)
    - 📌 Пример:  
    ```xml
<criterion comment="ruby is earlier than 0:2.5.3-104.module+el8.0.0+3250+4b7d6d43" test_ref="oval:com.redhat.rhsa:tst:20191972001"/>
```

🤔 <font color='brown'> Есть ли возможность упростить текущий формат, если да, то кратко
описать свой вариант для описания уязвимости вместе с проверками.  </font> 

### 🚀 Упрощаем формат
Я бы все перевел в формат JSON, но если необходим текущий формат, то есть два варианта:

- отталкиваясь от уязвимого пакета, а уже к нему тянуть все уязвимости в зависимости от версий
- отталкиваясь от бюллетени безопасности, а уже к ней тянуть все уязвимые пакеты.

Останавлюсь на **втором** варианте, как в оригинальном файле.
1) Уберу лишние критерии, про которые писал выше.
2) Добавлю новый тег <font color='brown'><b>&lt;vulnerable_packages&gt;</b></font>, который будет хранить теги <font color='brown'><b>&lt;vulnerable_package&gt;</b></font>, в которых будет содержатся наименование пакета, версия и оператор, по которому будет сравниваться версия. 
3) Удалю все критерии.
4) Удалю <font color='purple'><b>&lt;tests&gt;</b></font>,  <font color='red'><b>&lt;objects&gt;</b></font> и <font color='orange'><b>&lt;states&gt;</b></font>

##### Дополнительно:
Что-то менять/удалять в теге <font color='brown'><b>&lt;metadata&gt;</b></font> не вижу смысла, т.к. эта информация несет больше информативный характер и находится внутри <font color='green'><b>&lt;definitions&gt;</b></font>, но если это потребовалось, то что бы я сделал: 
- разбил "cvss3" на отдельные score и vector (как вариант, можно было разбить и на третью переменную - "версия cvss", но в данном документе используется только третья версия.
- убрал <font color='brown'><b>&lt;reference&gt;</b></font> для всех CVE, т.к. данную информацию дублирует тег <font color='brown'><b>&lt;cve&gt;</b></font>
- поменял ключ <font color='brown'><b>&lt;affected&gt;</b></font>, чтобы он содержал актуальные данные о двух продуктах и мог динамически изменяться, при добавлении нового продукта (как это случилось 20 апреля 2021 года)
- удалил весь тег <font color='brown'><b>&lt;affected_cpe_list&gt;</b></font>, т.к. никакой полезной информации в себе не несет и может только запутаь аналитика/заказчика
- добавил тег <font color='brown'><b>&lt;security_id&gt;</b></font>, в котором содержался бы уникальный идентификатор бюллетени для более быстрого поиска в документе. Сам идентификатор можно получить из заголовка (сплитанув по ":" или из ref_id внутри <font color='brown'><b>&lt;reference&gt;</b></font>

🤔 <font color='brown'> После выполненного в предыдущей пунктах анализа, необходимо
разработать приложение (скрипт) на языке Python, которое произведет
разбор (парсинг) OVAL-файла (достаточно сделать только первые 3 и
связанными с ними объекты) и преобразует его в ваш упрощенный
формат. </font> 

### 🚀 Скрипт (парсер по файлу)

In [1]:
import xml.etree.ElementTree as ET


#Файл rhel-8.oval.xml должен находится в папке с исполняемым кодом или пропишите свой путь
tree = ET.parse('rhel-8.oval.xml')
root = tree.getroot()

#Удаление неймспейсов
for k in root.iter():
    if '}' in k.tag:
        k.tag = k.tag.split('}', 1)[1]
        
#Обработка первых трех <definition> из <definitions> (первые три бюллетени)
definitions_elem = root.find('definitions')
if definitions_elem is not None:
    definition_all = definitions_elem.findall('definition')
    #Можно убрать, чтобы пройтись по всему файлу (по условию задачи оставил первые 3)
    for i in definition_all[3:]:
        definitions_elem.remove(i)
        
#Словари для заполнение нового тега <vulnerable_packages>
tests_map = {}
objects_map = {}
states_map = {}

#Заполнение tests_map для всех rpminfo_test  
'''Есть еще вид"ind-def:textfilecontent54_test", но к нему не стал дописывать проверку, 
т.к. для первых трех он не используется. А так можно сделать еще один тег, который будет 
хранить путь к файлу и по регулярному выражению из state искать совпадение в файле.
Соответственно для object и state подобный формат тоже не обрабатывается.'''

tests = root.find('tests')
if tests is not None:
    for t in tests.findall('rpminfo_test'):
        object_ref = None
        state_ref = None
        object_elem = t.find('object')
        state_elem = t.find('state')
        if object_elem is not None:
            object_ref = object_elem.get('object_ref')
        if state_elem is not None:
            state_ref = state_elem.get('state_ref')
        test_id = t.get('id')
        if test_id:
            tests_map[test_id] = (object_ref, state_ref)
            
#Заполнение objects_map для всех rpminfo_object
objects = root.find('objects')
if objects is not None:
    for ob in objects.findall('rpminfo_object'):
        object_id = ob.get('id')
        name_elem = ob.find('name')
        if object_id and name_elem is not None:
            objects_map[object_id] = name_elem.text
            
#Заполнение states_map для всех rpminfo_state (берем только operator и версию из evr)
states_section = root.find('states')
if states_section is not None:
    for s in states_section.findall('rpminfo_state'):
        state_id = s.get('id')
        evr_elem = s.find('evr')
        if state_id and evr_elem is not None:
            # Логический оператор и значение версии (EVR)
            states_map[state_id] = (evr_elem.get('operation'), evr_elem.text)
            
#Обработка первых трех бюллетеней
for def_elem in definitions_elem.findall('definition'):
    
    #Новый элемент <vulnerable_packages>
    vuln_packages_elem = ET.Element('vulnerable_packages')

    #Цикл по сбору всех критериев
    for crit in def_elem.findall('.//criterion'):
        test_ref = crit.get('test_ref')
        if not test_ref:
            continue
        #Не рассматриваю критерии, если тест отсутствует в rpminfo_test
        if test_ref not in tests_map:
            continue
        object_ref, state_ref = tests_map[test_ref]
        #Получение названия пакета по object_ref
        package_name = objects_map.get(object_ref, "")
        
        #Получение лог. оператора и версии
        operator = None
        version = None
        if state_ref and state_ref in states_map:
            operator, version = states_map[state_ref]
        else:
            continue

        #Новый тег, который будет хранить - пакет, его версию и лог. оператор
        vp_elem = ET.Element('vulnerable_package')
        vp_elem.set('product', package_name)
        if operator:
            vp_elem.set('operator', operator)
        if version:
            vp_elem.set('version', version)
        vuln_packages_elem.append(vp_elem)
        
    #Удаление критериев
    crit_elem = def_elem.find('criteria')
    if crit_elem is not None:
        def_elem.remove(crit_elem)
        
    #Добавление нового тега
    def_elem.append(vuln_packages_elem)
    
#Удаление лишних главных объектов
root.remove(root.find('tests'))
root.remove(root.find('objects'))
root.remove(root.find('states'))


#Сохранение результатов:
# Сохранение в новый файл
tree.write('itog_file_with_3_bull.oval.xml', encoding='utf-8', xml_declaration=True)

#Вывод результатов:
ET.indent(root, space="    ")
print("Мой более упрощенный вариант, где вся информация по бюллетеням находится в одном месте:\n\n\n")
print(ET.tostring(root, encoding="unicode"))

Мой более упрощенный вариант, где вся информация по бюллетеням находится в одном месте:



<oval_definitions xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://oval.mitre.org/XMLSchema/oval-common-5 oval-common-schema.xsd http://oval.mitre.org/XMLSchema/oval-definitions-5 oval-definitions-schema.xsd http://oval.mitre.org/XMLSchema/oval-definitions-5#unix unix-definitions-schema.xsd http://oval.mitre.org/XMLSchema/oval-definitions-5#linux linux-definitions-schema.xsd">
    <generator>
        <product_name>Red Hat OVAL Patch Definition Merger</product_name>
        <product_version>3</product_version>
        <schema_version>5.10</schema_version>
        <timestamp>2025-03-07T06:30:14</timestamp>
        <content_version>1741329014</content_version>
    </generator>
    <definitions>
        <definition class="patch" id="oval:com.redhat.rhba:def:20191992" version="635">
            <metadata>
                <title>RHBA-2019:1992: cloud-init bug fix and enh